# per ogni input controllare che l'ufficio sia presente nella lista dei codici uffici

#### Inseriamo ufficio ( tramite id )
#### Inseriamo ente ( tramite id ) numero, anno , ricerca_iqera 
#### per ora entriamo solo nella pagina non facciamo lo scraper dei risultati

In [42]:
import requests
from bs4 import BeautifulSoup

In [43]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if 'non trovato' in response2.text:
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','no']
            else :
                compila = 'no'
                tabella = soup2.find('div', class_='panel-body')

                if  tabella.find('span', string="Atto tassato dall'ufficio di: "): 
                    office = tabella.find('span', string="Atto tassato dall'ufficio di: ").find_next_sibling('span').get_text(strip=True)
                else :
                    office = '-'   

                if tabella.find('span', string="Tipologia atto: "):
                    atto = tabella.find('span', string="Tipologia atto: ").find_next_sibling('span').get_text(strip=True)
                else :
                    atto = '-'  

                if tabella.find('span', string="Emesso da: "):
                    emesso_da = tabella.find('span', string="Emesso da: ").find_next_sibling('span').get_text(strip=True)
                else :
                    emesso_da = '-'   

                if tabella.find('span', string="n. Repertorio - anno: "):
                    repertorio = tabella.find('span', string="n. Repertorio - anno: ").find_next_sibling('span').get_text(strip=True)
                else :
                    repertorio = '-'  

                if tabella.find('span', string="Attore: "):
                    attore = tabella.find('span', string="Attore: ").find_next_sibling('span').get_text(strip=True)
                else:
                    attore='-'

                if tabella.find('span', string="Convenuto: "):
                    convenuto = tabella.find('span', string="Convenuto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    convenuto ='-'

                if tabella.find('span', string="Stato dell'atto: "):
                    stato = tabella.find('span', string="Stato dell'atto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    stato= '-'

                if 'compilaf24.php' in response2.text:  # Check if the link is present in the whole body
                    compila = 'si'



                return [office,atto,emesso_da,repertorio,attore,convenuto,stato,compila]
        else :
            rep = str(numero) + ' - ' + str(anno)
            return [ufficio, natura, ente, rep, '-', '-', '-','no']

    else:
       return 'ufficio non trovato'   

In [44]:
"""office,atto,emesso_da,repertorio,attore,convenuto,stato = scraper(ufficio='TQD',ente='002',anno='2017',numero='3',natura='DI')
prova = {
    'ufficio':office,
    'atto':atto,
    'emesso da':emesso_da,
    'repertorio':repertorio,
    'attore':attore,
    'convenuto':convenuto,
    'stato':stato
}
prova"""

"office,atto,emesso_da,repertorio,attore,convenuto,stato = scraper(ufficio='TQD',ente='002',anno='2017',numero='3',natura='DI')\nprova = {\n    'ufficio':office,\n    'atto':atto,\n    'emesso da':emesso_da,\n    'repertorio':repertorio,\n    'attore':attore,\n    'convenuto':convenuto,\n    'stato':stato\n}\nprova"

#### cambia la natura dell'ente in base al tipo diu ricerca ( se monitoria o ppt)

In [45]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

def tipo_scraper(contratto,debitore, tipo, ufficio, lista_enti, anno, numero,row_number):
    results = []
    try:
        if tipo == 'monitoria':
            natura_list = list_natura_monitoria
        elif tipo == 'ppt':
            natura_list = list_natura_ppt
        else:
            raise ValueError("Invalid tipo value")

        for natura in natura_list:
            for ente in lista_enti:
                result = scraper(ufficio, ente, anno, numero, natura)
                if isinstance(result, list):
                    if len(result) == 8:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'atto': result[1],
                            'emesso da': result[2],
                            'repertorio': result[3],
                            'attore': result[4],
                            'convenuto': result[5],
                            'stato': result[6],
                            "compila":result[7]
                        })
                    else:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'natura': result[1],
                            'ente': result[2],
                            'repertorio': result[3],
                            'attore': '-',
                            'convenuto': '-',
                            'stato': '-',
                            "compila":'no'
                        })
                else:
                    print(result)
    except Exception as e:
        print(f'Riga con errore: {row_number}')
        print(f"Error occurred: {e}")
    return results


#### sistemiamo l'input prima di mandare lo scaper 

In [46]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
import numpy as np
today = date.today()


In [47]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [48]:
input = pd.read_excel('Input/Raw/perimetro_27_05.xlsx')

In [49]:
n_columns = input.filter(regex='^N')
contains_dash = n_columns.applymap(lambda x: '-' in str(x) and '/' not in str(x))
n_columns = n_columns.applymap(lambda x: x.replace('-', '/') if '-' in str(x) else x)
input[n_columns.columns] = n_columns

In [50]:

input = clean(input)
#input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
#input = input.drop(columns=['RICERCA IQERA'])

mapping = {'albenga':'savona',
 'chiavenna':'sondrio',
 'cossato':'biella',
 'macomer':'nuoro',
 'isili':'nuoro',
 'chiaromonte': 'lagonegro',
 'orzinuovi' : 'brescia',
 'ovada': 'acquiterme',
 'pavullo nel frignano':'modena',
 'ponte san pietro':'bergamo',
 'romano di lombardia':'bergamo',
 'san benedetto del tronto':'ascoli piceno',
 'teano':'caserta',
 'venezia 2 - mestre' : 'venezia',
 'venosa':'potenza',
 'vimercate':'monza',
 'acerra' : 'casoria',
 'torre annunziata':'napoli',
 'napoli nord in aversa':'napoli',
 'napoli nord':'napoli',
 'san giorgio ionico':'taranto'

 }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input


In [51]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [52]:

input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

In [53]:
input[input['code_office'].isna()]['Foro'].drop_duplicates()

Series([], Name: Foro, dtype: object)

#### sopra abbiamo aggiunto il codice dell'ufficio

#### ora dobbiamo aggiungere i codici degli enti

In [54]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [55]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)

In [56]:
# Convert the 'code_ente' lists to strings
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))


potential_value_vars = [
    'nr.decreto', 'year.decreto', 'nr.rg', 'year.rg',
    'nr.repertorio', 'year.repertorio', 'nr.crono', 'year.crono',
    'nr.rgeppt', 'year.rgeppt', 'nr.repppt', 'year.repppt'
]
value_vars = [col for col in potential_value_vars if col in input.columns]
# Melt the DataFrame to 'pivot longer'
df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=value_vars, 
                  var_name='type', 
                  value_name='value')

# Split the 'type' column into 'attribute' and 'category'
df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

# Pivot the DataFrame to get the desired format
input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

# Flatten the columns
input.columns.name = None
input = input.reset_index(drop=True)

# Convert the 'code_ente' column back into a list
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [57]:
input.to_excel('Input/Cleaned/2024_05_27_input_v1.xlsx',index=False)

In [58]:
results =( input
    .apply(lambda row: tipo_scraper(contratto=row['NR_Contratto'],
                                    debitore=row['Debitore'],
                                    tipo= row['tipo_ricerca'],
                                    ufficio= row['code_office'],
                                    lista_enti=row['code_ente'], anno=row['year'],numero= row['nr'],
                                    row_number=row.name
                                    ),
                                    axis=1)
)

In [59]:
results

0      [{'contratto': 'TBCC0100353363.001', 'debitore...
1      [{'contratto': 'TBCC0100353363.001', 'debitore...
2      [{'contratto': 'TBCC0100700107.001', 'debitore...
3      [{'contratto': 'TBCC0100700107.001', 'debitore...
4      [{'contratto': 'TBCC0100700107.001', 'debitore...
                             ...                        
307    [{'contratto': 'TBS8631548', 'debitore': 'THIA...
308    [{'contratto': 'TBS8631548', 'debitore': 'THIA...
309    [{'contratto': 'TBS8631548', 'debitore': 'THIA...
310    [{'contratto': 'XTB65236651', 'debitore': 'MIN...
311    [{'contratto': 'XTB65236651', 'debitore': 'MIN...
Length: 312, dtype: object

In [60]:
filtered_data = [item for item in results if item != []]
flattened_results = [item for sublist in filtered_data for item in (sublist if isinstance(sublist, list) else [sublist])]
df = pd.DataFrame(flattened_results)
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
0,TBCC0100353363.001,FERRARI CRISTINA,BRESCIA UT DPBS,ESECUZIONE MOBILIARE n. 1819 del 2021,TRIBUNALE DI BRESCIA,862/22 - 2022,TIBERIUS SPV S.R.L.,FERRARI CRISTINA,EMESSO ARTICOLO DI RUOLO,no
1,TBCC0100353363.001,FERRARI CRISTINA,TM4,EM,004,1819 - 2021,-,-,-,no
2,TBCC0100353363.001,FERRARI CRISTINA,TM4,EM,011,1819 - 2021,-,-,-,no
3,TBCC0100353363.001,FERRARI CRISTINA,TM4,OR,001,1819 - 2021,-,-,-,no
4,TBCC0100353363.001,FERRARI CRISTINA,TM4,OR,004,1819 - 2021,-,-,-,no
...,...,...,...,...,...,...,...,...,...,...
4381,XTB65236651,MINUTILLI IVANO,TJK,AV,003,191 - 2023,-,-,-,no
4382,XTB65236651,MINUTILLI IVANO,TJK,DI,001,5561 - 2022,-,-,-,no
4383,XTB65236651,MINUTILLI IVANO,TJK,DI,003,5561 - 2022,-,-,-,no
4384,XTB65236651,MINUTILLI IVANO,TJK,AV,001,5561 - 2022,-,-,-,no


In [61]:
df[(df['attore'] != '-') | (df['convenuto'] != '-') ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
0,TBCC0100353363.001,FERRARI CRISTINA,BRESCIA UT DPBS,ESECUZIONE MOBILIARE n. 1819 del 2021,TRIBUNALE DI BRESCIA,862/22 - 2022,TIBERIUS SPV S.R.L.,FERRARI CRISTINA,EMESSO ARTICOLO DI RUOLO,no
92,TBCO0010259378.074,BIONDO ANTONINO,MESSINA UT DPME,DECRETO INGIUNTIVO n. 1669 del 2022,TRIBUNALE DI MESSINA,791/2023 - 2022,BLADE MANAGEMENT S.R.L.,-,EMESSO ARTICOLO DI RUOLO,no
167,TBCO0010834315.074,VULIN MILORAD,TRIESTE UT DPTS,ORDINANZA n. 636 del 2023,TRIBUNALE DI TRIESTE,636 - 2023,PENZO DORIA GIANNI,PAOLUZZI ALESSANDRO E ANDREA,IN ATTESA DI PAGAMENTO,si
186,TBCO0010930318.074,VILLARI MARCO VINCENZO,CATANIA UT DPCT,DECRETO INGIUNTIVO n. 5159 del 2021,TRIBUNALE DI CATANIA,5334/23 - 2021,TIBERIUS SPV S.R.L.,-,ATTO IN FASE DI TASSAZIONE,no
235,TBCO0011119692.074,PEPE' SCIARRIA MARCO,CIVITAVECCHIA UT DPRM2,ATTO VARIO n. 822 del 2021,TRIBUNALE DI CIVITAVECCHIA,822/21 - 2021,DE MICHELIS,DE MUTIIS,ATTO REGISTRATO,no
...,...,...,...,...,...,...,...,...,...,...
4128,TBS81966419,TERRACCIANO STEFANO,NAPOLI 3 UT DPNA2,ATTO VARIO n. 770 del 2021,TRIBUNALE DI NAPOLI,770 - 2021,HOOVER,COMMISSIONE TERRITORIALE PER I,"ATTO REGISTRATO A DEBITO, ATTENZIONE: non effe...",no
4142,TBS81966419,TERRACCIANO STEFANO,NAPOLI 3 UT DPNA2,DECRETO INGIUNTIVO n. 4904 del 2021,TRIBUNALE DI NAPOLI,908 - 2021,KIMBO SPA,NEAPOLITAN COOKING S.R.L.,ATTO REGISTRATO,no
4348,TBS8631548,THIAM CHEIKH,VENEZIA 1 UT DPVE,ATTO VARIO n. 40 del 2021,TRIBUNALE DI VENEZIA,40 - 2021,-,SIPIL S.R.L.,ATTO REGISTRATO,no
4351,TBS8631548,THIAM CHEIKH,VENEZIA 1 UT DPVE,ATTO VARIO n. 40 del 2021,REPERTORIO SICC. TRIB. VE,40 - 2021,KHAN,MINISTERO DELL' INTERNO,"ATTO REGISTRATO A DEBITO, ATTENZIONE: non effe...",no


In [64]:
df[(df['attore'].str.contains('\+')) | (df['convenuto'].str.contains('\+')) ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila


In [ ]:
mask_attore = df['attore'].str.contains('\+')
mask_convenuto = df['convenuto'].str.contains('\+')

df.loc[mask_attore, 'attore'] = df.loc[mask_attore, 'attore'].str.split('\+').str[0].str.strip()
df.loc[mask_convenuto, 'convenuto'] = df.loc[mask_convenuto, 'convenuto'].str.split('\+').str[0].str.strip()


In [62]:
# df = pd.DataFrame(results.explode().tolist())
df.to_excel(f'Output/{str(today)}_output_totale_v1.xlsx',index=False)


In [65]:
mask = df.apply(lambda row: (row['attore'] in row['debitore']) or (row['convenuto'] in row['debitore']), axis=1)

df[mask].to_excel(f'Output/{str(today)}_esiti_positivi_v1.xlsx',index=False)